Первое знакомство с данными

In [113]:
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.svm import SVC, LinearSVC
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score
from scipy.sparse import csr_matrix, hstack, vstack, csc_matrix
from nltk.stem import PorterStemmer
import scipy as sc
from sklearn.preprocessing import OneHotEncoder
from mlxtend.classifier import StackingClassifier, StackingCVClassifier
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
import nltk
%matplotlib inline

In [114]:
train = pd.read_csv('task2_lemmas_train.txt', names=['Id', 'X', 'y'])
test = pd.read_csv('task2_lemmas_test.txt', names=['Id', 'X'])

In [115]:
train[:10]

,Id,X,y
0,Id,X,y
1,1,vergognerete,vergognare+V
2,2,amnistiavate,amnistiare+V
3,3,menomazione,menomazione+N
4,4,sfaldavamo,sfaldare+V
5,5,sfodererei,sfoderare+V
6,6,ascondesti,ascondere+V
7,7,edifichereste,edificare+V
8,8,maschieran,maschiare+V
9,9,transennasser,transennare+V


In [116]:
train = train.drop(['Id'], axis=1)
train = train.drop(0, axis=0)
test = test.drop(['Id'], axis=1)
test = test.drop(0, axis=0)

In [117]:
train[10:20]

,X,y
11,accudisser,accudire+V
12,diromperanno,dirompere+V
13,intercollegai,intercollegare+V
14,integrando,integrare+V
15,sbramerebbero,sbramare+V
16,stravaccherebbero,stravaccare+V
17,oltrepassasser,oltrepassare+V
18,cauzionavan,cauzionare+V
19,scarrozzata,scarrozzare+V
20,intossicassero,intossicare+V


In [118]:
test[:3]

,X
1,gettonan
2,incidentali
3,involtino


In [119]:
from nltk.stem import PorterStemmer
from nltk.stem.lancaster import LancasterStemmer

In [120]:
ps = PorterStemmer()
ps2 = LancasterStemmer()
for word in train.X[:10]: 
    print word, "->", ps.stem(word)
    print word, "->", ps2.stem(word)

vergognerete -> vergogneret
vergognerete -> vergogneret
amnistiavate -> amnistiav
amnistiavate -> amnistiav
menomazione -> menomazion
menomazione -> menomaz
sfaldavamo -> sfaldavamo
sfaldavamo -> sfaldavamo
sfodererei -> sfodererei
sfodererei -> sfoderere
ascondesti -> ascondesti
ascondesti -> ascondest
edifichereste -> edificherest
edifichereste -> edificherest
maschieran -> maschieran
maschieran -> maschy
transennasser -> transennass
transennasser -> transennass
computando -> computando
computando -> computando


In [121]:
from nltk.tokenize import word_tokenize

In [122]:
text = word_tokenize("computare ")
nltk.pos_tag(text)

[('computare', 'NN')]

In [123]:
from nltk.stem import WordNetLemmatizer

In [124]:
wordnet_lemmatizer = WordNetLemmatizer()
wordnet_lemmatizer.lemmatize('transennasser')

'transennasser'

In [125]:
form = []
for i in train.y:
    form.append(i[-1])

In [126]:
form[:10]

['V', 'V', 'N', 'V', 'V', 'V', 'V', 'V', 'V', 'V']

In [127]:
train['form'] = form

In [128]:
train[:10]

,X,y,form
1,vergognerete,vergognare+V,V
2,amnistiavate,amnistiare+V,V
3,menomazione,menomazione+N,N
4,sfaldavamo,sfaldare+V,V
5,sfodererei,sfoderare+V,V
6,ascondesti,ascondere+V,V
7,edifichereste,edificare+V,V
8,maschieran,maschiare+V,V
9,transennasser,transennare+V,V
10,computando,computare+V,V


In [129]:
print (train.loc[(train['form'] =='N')].size+train.loc[(train['form'] =='A')].size)/(train.loc[(train['form'] =='V')].size.astype(float)+train.loc[(train['form'] =='N')].size+train.loc[(train['form'] =='A')].size) 

0.191958867161


In [187]:
train.size

593200

In [130]:
import re

In [131]:
train['word'] = train.y.map(lambda x: re.sub("[+,0-9, A-Z]", "", x))

In [132]:
train[:5]

,X,y,form,word
1,vergognerete,vergognare+V,V,vergognare
2,amnistiavate,amnistiare+V,V,amnistiare
3,menomazione,menomazione+N,N,menomazione
4,sfaldavamo,sfaldare+V,V,sfaldare
5,sfodererei,sfoderare+V,V,sfoderare


In [133]:
train = train.drop(['y'], axis=1)

In [134]:
train[:1]

,X,form,word
1,vergognerete,V,vergognare


In [163]:
w1 = 'lalasadff'
w2 = 'lalkkk'

In [174]:
def base_and_ending(X, y):
    base = []
    ending = []
    flag = True
    if len(X) >= len(y):
        length = len(y)
    else:
        length = len(X)
    for i in range(length):
        if X[i] == y[i] and flag==True:
            base.append(X[i])
        else:
            flag=False
            ending.append(y[i])
    return ''.join(base), ''.join(ending)

In [175]:
base_and_ending(w1, w2)

('lal', 'kkk')

In [178]:
base_and_ending('vergognerete', 'vergognare')
base_and_ending('sfodererei', 'sfoderare')
base_and_ending('ascondesti', 'ascondere')

('asconde', 're')

In [180]:
base = []
ending = []
for X, y in zip(train.X, train.word):
    b, e = base_and_ending(X, y)
    base.append(b)
    ending.append(e)

In [181]:
train['base'] = base
train['ending'] = ending

In [182]:
train[:20]

,X,form,word,base,ending
1,vergognerete,V,vergognare,vergogn,are
2,amnistiavate,V,amnistiare,amnistia,re
3,menomazione,N,menomazione,menomazione,
4,sfaldavamo,V,sfaldare,sfalda,re
5,sfodererei,V,sfoderare,sfoder,are
6,ascondesti,V,ascondere,asconde,re
7,edifichereste,V,edificare,edific,are
8,maschieran,V,maschiare,maschi,are
9,transennasser,V,transennare,transenna,re
10,computando,V,computare,computa,re


In [355]:
train.ending.value_counts()[:]

re             44200
are            24023
               17800
e               8047
o               6792
a               4353
r               3357
ar              3098
iare            1765
ere             1191
rre              756
ire              756
er               472
ir               378
i                287
ge               124
ore              122
nde               73
nd                71
rr                60
nire              60
nere              55
gge               51
de                49
uocere            47
lere              40
ger               31
nder              29
tte               27
gere              24
               ...  
ole                1
a-ordinanza        1
ote                1
desistere          1
ter                1
es                 1
onde               1
o-radar            1
a-partito          1
ler                1
x                  1
o-stato            1
e-quadro           1
a-gol              1
ompere             1
buono              1
esse         

In [188]:
train.form.value_counts()

V    95866
A    13166
N     9608
Name: form, dtype: int64

Предсказав re можно легко из исходного слова получить нужное - достаточно просто идти с конца и достигнув первой буквы а, добавить re. Исследуем данные дальше, чтобы посмотреть, что ещё можно выкинуть.

In [194]:
train.loc[(train['ending']=='re')]

,X,form,word,base,ending
2,amnistiavate,V,amnistiare,amnistia,re
4,sfaldavamo,V,sfaldare,sfalda,re
6,ascondesti,V,ascondere,asconde,re
9,transennasser,V,transennare,transenna,re
10,computando,V,computare,computa,re
11,accudisser,V,accudire,accudi,re
14,integrando,V,integrare,integra,re
17,oltrepassasser,V,oltrepassare,oltrepassa,re
18,cauzionavan,V,cauzionare,cauziona,re
19,scarrozzata,V,scarrozzare,scarrozza,re


In [196]:
train.base.unique().size, train.X.unique().size

(26226, 118640)

In [216]:
train = train.drop(['word', '1'], axis=1)

In [217]:
train[:3]

,X,form,base,ending
1,vergognerete,V,vergogn,are
2,amnistiavate,V,amnistia,re
3,menomazione,N,menomazione,


In [236]:
end_length = []
for X, base in zip(train.X, train.base):
    end_length.append(len(X.replace(base,"")))

In [237]:
train['end_l'] = end_length

In [238]:
train[:10]

,X,form,base,ending,end_l
1,vergognerete,V,vergogn,are,5
2,amnistiavate,V,amnistia,re,4
3,menomazione,N,menomazione,,0
4,sfaldavamo,V,sfalda,re,4
5,sfodererei,V,sfoder,are,4
6,ascondesti,V,asconde,re,3
7,edifichereste,V,edific,are,7
8,maschieran,V,maschi,are,4
9,transennasser,V,transenna,re,4
10,computando,V,computa,re,3


In [239]:
base = train['base']
X = train['X']

In [241]:
train = train.drop(['base'], axis=1)

In [243]:
train[:2]

,X,form,ending,end_l
1,vergognerete,V,are,5
2,amnistiavate,V,re,4


In [245]:
train.end_l.unique()

array([ 5,  4,  0,  3,  7,  1,  8,  9,  2,  6, 10, 11, 12], dtype=int64)

In [246]:
train.form.unique()

array(['V', 'N', 'A'], dtype=object)

In [255]:
end_l_fact = pd.factorize(train.end_l)[0]
ending_fact = pd.factorize(train.end_l)[0]

In [256]:
end_f[:10]

array([0, 1, 2, 1, 1, 3, 4, 1, 1, 3], dtype=int64)

In [257]:
ending_fact[:10]

array([0, 1, 2, 1, 1, 3, 4, 1, 1, 3], dtype=int64)

In [259]:
train_new = pd.DataFrame(train)

In [313]:
train['ending_fact']=ending_fact

In [265]:
train = train.drop(['1'], axis=1)

In [284]:
CountVect.fit_transform(train.X[:10]).toarray()

array([[2, 0, 0, ..., 0, 0, 0],
       [2, 1, 1, ..., 0, 0, 0],
       [2, 0, 0, ..., 1, 1, 1],
       ..., 
       [2, 0, 0, ..., 0, 0, 0],
       [2, 0, 0, ..., 0, 0, 0],
       [2, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [361]:
CountVect = CountVectorizer(analyzer='char_wb', ngram_range=(1, 15))

In [362]:
X = CountVect.fit_transform(train.X)
#X = X.toarray()
y=train['ending_fact']
print X.shape

(118640, 1466722)


In [279]:
model_lr = LogisticRegression()

In [281]:
model_lr.fit(X=X, y=ending_fact)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

ngram_range = (1, 2)

In [282]:
cross_val_score(model_lr, X=X, y=ending_fact, scoring='accuracy')

C:\Users\Petr\Anaconda2\lib\site-packages\sklearn\model_selection\_split.py:581: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of groups for any class cannot be less than n_splits=3.
  % (min_groups, self.n_splits)), Warning)


array([ 0.71134203,  0.71430739,  0.71217945])

ngram_range=(1, 3)

In [286]:
cross_val_score(model_lr, X=X, y=ending_fact, scoring='accuracy')

array([ 0.8625354 ,  0.86365246,  0.86411916])

ngram_range=(1,4)

In [315]:
cross_val_score(model_lr, X=X, y=y, scoring='accuracy')

array([ 0.85826175,  0.85060667,  0.85392405])

ngram_range=(1,5)

In [318]:
cross_val_score(model_lr, X=X, y=y, scoring='accuracy')

array([ 0.86609399,  0.85768453,  0.86177215])

ngram_range=(1, 6)

In [322]:
cross_val_score(model_lr, X=X, y=y, scoring='accuracy')

array([ 0.8676099 ,  0.85667341,  0.86658228])

ngram_range=(1, 7)

In [333]:
cross_val_score(model_lr, X=X, y=y, scoring='accuracy')

array([ 0.86457807,  0.85439838,  0.86607595])

ngram_range=(1, 10)

In [354]:
print X.shape
cross_val_score(model_lr, X=X, y=y, scoring='accuracy')

(118640, 907573)


array([ 0.92420105,  0.92383553,  0.92580041])

ngram_range=(1, 11)

In [358]:
print X.shape
cross_val_score(model_lr, X=X, y=y, scoring='accuracy')

(118640, 1093627)


array([ 0.92369539,  0.92322865,  0.92514289])

ngram_range=(1,15)

In [363]:
print X.shape
cross_val_score(model_lr, X=X, y=y, scoring='accuracy')

(118640, 1466722)


array([ 0.92288633,  0.9223436 ,  0.92418188])